In [1]:
import pandas as pd
import requests
import re
import numpy as np
import ast
import pickle

In [2]:
df = pd.read_csv("final_dataset_with_20000_movies.csv", index_col=0)

In [3]:
df.sample(5)

,movie_id,movie_name,genre,overview,director,cast,Keywords,keywords
2334,tt0079590,Muqaddar Ka Sikandar,"Romance, Drama, Action","Sikandar, an orphan, makes many friends and fo...",Prakash Mehra,"Amitabh Bachchan, Vinod Khanna, Rakhee Gulzar,...",NaN,NaN
930,tt0062670,Anjaam,"Action, Adventure, Drama",Shekhar (Feroz Khan) helps C.B.I agent Rekha (...,Shiv Kumar,"Feroz Khan, Shahida, Nisar Ahmad Ansari, Azaad...",NaN,NaN
1707,tt1427097,Ponnunnjal,NaN,"Film starring Shivaji Ganesan, Muthuraman and ...",NaN,NaN,NaN,NaN
8616,tt0354405,Bas Yun Hi,"Music, Comedy, Romance",Comedy - Aditya is a young man who does not be...,Raja Menon,"Purab Kohli, Nandita Das, Rajiv Gopalakrishnan...",NaN,NaN
11064,tt2359746,Pancharangi,"Drama, Comedy",A family goes to a village near Mangalore to f...,Yogaraj Bhat,"Pawan Kumar, Nidhi Subbaiah, Anant Nag, Ramya ...",NaN,arranged marriage


In [4]:
df['key_words'] = df[['Keywords', 'keywords']].apply(lambda x: ', '.join([str(i) for i in x if pd.notna(i)]), axis=1)

In [5]:
df = df.drop(columns=["Keywords","keywords"])

In [6]:
df.rename(columns={"key_words":"keywords"}, inplace=True)

In [7]:
df.sample(10)

,movie_id,movie_name,genre,overview,director,cast,keywords
2212,tt3083968,Zee Boom Ba,Horror,Indian horror movie from 1977.,NaN,"Jayashankar, Jyothi Lakshmi, Vijayalalitha",
3129,tt27815015,Kurukku,"Crime, Mystery, Thriller",A police officer who has to investigate and un...,Abhijith Noorani,"Ajayaghosh, Anil Anto, Sabari Chandran, Sandee...","police, murder"
20285,tt14209750,Bhala Thandanana,"Drama, Action",An investigative journalist digs deeper into a...,Chaitanya Dantuluri,"Sree Vishnu, Catherine Tresa, Ramachandra Raju...",
22888,tt27198438,Jenni,NaN,A 9 years boy gets a traditional wooden toy of...,B.Nithianandam,NaN,
2344,tt0272270,Randilonnu,Drama,A woman adopts the abandoned child of another ...,A. S. Pragasam,"Sukumaran, Ravi Menon, Venniradai Nirmala, Sad...",
2366,tt0236977,Amar Shakti,"Action, Adventure, Comedy","In order to take over the kingdom, Shamsher Si...",Harmesh Malhotra,"Shashi Kapoor, Shatrughan Sinha, Jeevan Dhar, ...",
1414,tt0072100,Roti Kapada Aur Makaan,"Action, Drama, Musical",Bharat faces challenges in leading his family ...,Manoj Kumar,"Manoj Kumar, Shashi Kapoor, Zeenat Aman, Moush...","labor strike, brother brother relationship, fa..."
1030,tt0150115,Chiradiner,Drama,A 1969 Bengali Drama Film directed by Agradoot...,Agradoot,"Uttam Kumar, Supriya Choudhury, Kamal Mitra, B...",
3387,tt0309764,Kaviratna Kalidasa,"Drama, History, Music","Sanskrit and upper-caste poet, Kalidas, faces ...",Renuka Sharma,"Dr. Rajkumar, Srinivasa Murthy, Jaya Prada",
636,tt0060310,Dil Diya Dard Liya,"Drama, Romance","A kind-hearted, widowed landlord, Thakur, live...","Dilip Kumar, Abdul Rashid Kardar","Dilip Kumar, Waheeda Rehman, Pran Sikand, Rehm...",


In [8]:
df.isnull().sum()

movie_id         0
movie_name       0
genre         2871
overview       812
director      1172
cast          1153
keywords         0
dtype: int64

In [8]:
# List of OMDb API keys to use as backups
omdb_api_keys = ['84058312', 'b931e931', '8ec0b50d', '224fce8b', '6724eaac', '9accbe5c', 'd6d2fd4c', '18e96f01', '34565ea7', 'ea03ff0e', '54b2a0ce', 'b2cd0aa9', '955cbf04', '38f6e58e', 'e0e493db',]  # Replace with your actual OMDb API keys

def fetch_from_omdb(imdb_id):
    """Fetch movie details from OMDb API with fallback to alternative keys if 401 Unauthorized error is encountered."""
    
    for omdb_api_key in omdb_api_keys:
        omdb_url = f"http://www.omdbapi.com/?i={imdb_id}&apikey={omdb_api_key}"
        
        try:
            response = requests.get(omdb_url)
            
            # Check if the response was successful
            if response.status_code == 200:
                data = response.json()
                
                # Check if the movie is found
                if data.get("Response") == "True":
                    return {
                        'imdb_id': data.get('imdbID', None),
                        'genre': data.get('Genre', None),
                        'overview': data.get('Plot', None),
                        'director': data.get('Director', None),
                        'cast': data.get('Actors', None),
                    }

            elif response.status_code == 401:
                print(f"API key {omdb_api_key} is unauthorized. Trying the next key...")
                continue  # Switch to the next API key
                
            else:
                print(f"Failed to fetch data for IMDb ID {imdb_id} with key {omdb_api_key}: {response.status_code}")
        
        except requests.exceptions.RequestException as e:
            print(f"Request failed for IMDb ID {imdb_id} with key {omdb_api_key}: {e}")
    
    # Return None if all keys are exhausted or if no data is found for the given IMDb ID
    print("All API keys have been exhausted or no data found for the given IMDb ID.")
    return None


In [9]:
def update_missing_data(indian_movies_df):
    """Update missing movie details in the DataFrame by fetching from OMDb."""
    for index, row in indian_movies_df.iterrows():
        # Check if any of the necessary fields are None
        if pd.isna(row['genre']) or pd.isna(row['overview']) or pd.isna(row['director']) or pd.isna(row['cast']):
            if row['movie_id']:
                omdb_data = fetch_from_omdb(row['movie_id'])

                # If data is returned from OMDb, update the DataFrame
                if omdb_data:
                    if pd.isna(row['genre']):
                        indian_movies_df.at[index, 'genre'] = omdb_data['genre']
                    if pd.isna(row['overview']):
                        indian_movies_df.at[index, 'overview'] = omdb_data['overview']
                    if pd.isna(row['director']):
                        indian_movies_df.at[index, 'director'] = omdb_data['director']
                    if pd.isna(row['cast']):
                        indian_movies_df.at[index, 'cast'] = omdb_data['cast']
                        
    return indian_movies_df

In [14]:
new_df = update_missing_data(df)

Failed to fetch data for IMDb ID tt1402532 with key 36067a5a: 522
All API keys have been exhausted or no data found for the given IMDb ID.
API key 36067a5a is unauthorized. Trying the next key...
API key 36067a5a is unauthorized. Trying the next key...
API key 36067a5a is unauthorized. Trying the next key...
API key 36067a5a is unauthorized. Trying the next key...
API key 36067a5a is unauthorized. Trying the next key...
API key 36067a5a is unauthorized. Trying the next key...
API key 36067a5a is unauthorized. Trying the next key...
API key 36067a5a is unauthorized. Trying the next key...
API key 36067a5a is unauthorized. Trying the next key...
API key 36067a5a is unauthorized. Trying the next key...
API key 36067a5a is unauthorized. Trying the next key...
API key 36067a5a is unauthorized. Trying the next key...
API key 36067a5a is unauthorized. Trying the next key...
API key 36067a5a is unauthorized. Trying the next key...
API key 36067a5a is unauthorized. Trying the next key...
API ke

In [15]:
new_df.isnull().sum()

movie_id        0
movie_name      0
genre         244
overview       79
director      139
cast          215
keywords        0
dtype: int64

In [20]:
new_df = update_missing_data(new_df)

API key 4a21f055 is unauthorized. Trying the next key...
API key 514778af is unauthorized. Trying the next key...
All API keys have been exhausted or no data found for the given IMDb ID.
API key de58fde is unauthorized. Trying the next key...
API key 514778af is unauthorized. Trying the next key...
All API keys have been exhausted or no data found for the given IMDb ID.
API key 4a21f055 is unauthorized. Trying the next key...
API key de58fde is unauthorized. Trying the next key...
All API keys have been exhausted or no data found for the given IMDb ID.
API key 4a21f055 is unauthorized. Trying the next key...
API key 514778af is unauthorized. Trying the next key...
All API keys have been exhausted or no data found for the given IMDb ID.
API key 4a21f055 is unauthorized. Trying the next key...
API key de58fde is unauthorized. Trying the next key...
All API keys have been exhausted or no data found for the given IMDb ID.
API key 4a21f055 is unauthorized. Trying the next key...
API key de5

In [65]:
new_df.to_csv('final_twentythousand_movie_dataset.csv', index=False)

In [206]:
dataframe = pd.read_csv("final_twentythousand_movie_dataset.csv")

In [207]:
dataframe.isnull().sum()

movie_id          0
movie_name        0
genre           901
overview        665
director         78
cast            386
keywords      15257
dtype: int64

In [208]:
new_df2 = update_missing_data(dataframe)

NameError: name 'update_missing_data' is not defined

In [209]:
new_df2.to_csv('final_twentythousand_movie_dataset2.csv', index=False)

NameError: name 'new_df2' is not defined

In [210]:
ddf=pd.read_csv("final_twentythousand_movie_dataset2.csv")

In [211]:
new_dataframe = ddf.copy()

In [212]:
new_dataframe.isnull().sum()

movie_id          0
movie_name        0
genre           561
overview        489
director         66
cast            324
keywords      15257
dtype: int64

In [213]:
new_dataframe.reset_index(drop=True, inplace=True)

In [214]:
new_dataframe

,movie_id,movie_name,genre,overview,director,cast,keywords
0,tt15354916,Jawan,"Action, Thriller",A high-octane action thriller which outlines t...,Atlee,"Shah Rukh Khan, Nayanthara, Vijay Sethupathi, ...","revenge, chase, suicide, farmer, train, injust..."
1,tt15748830,Jaane Jaan,"Crime, Drama, Mystery",A single mother and her daughter who commit a ...,Sujoy Ghosh,"Kareena Kapoor, Jaideep Ahlawat, Vijay Varma, ...","single mother, police, investigation, criminal..."
2,tt11663228,Jailer,"Action, Comedy, Crime",A retired jailer goes on a manhunt to find his...,Nelson Dilipkumar,"Rajinikanth, Mohanlal, Shivarajkumar, Jackie S...","prison, ex cop, intermission, action hero, one..."
3,tt14993250,Rocky Aur Rani Kii Prem Kahaani,"Comedy, Drama, Family",Flamboyant Punjabi Rocky and intellectual Beng...,Karan Johar,"Ranveer Singh, Alia Bhatt, Dharmendra, Shabana...","love, family relationships, rom com, news repo..."
4,tt15732324,OMG 2,"Comedy, Drama",An unhappy civilian asks the court to mandate ...,Amit Rai,"Pankaj Tripathi, Akshay Kumar, Yami Gautam, Pa...","courtroom, sex education, court room drama, re..."
...,...,...,...,...,...,...,...
20829,tt32623558,Black Coffee,Drama,Film director Tara discovers her sister Vanya'...,Khan Aamir,"Richa Singh Chauhan, Mamta Jain, Sachin Vidroh...",NaN
20830,tt32361752,Hues & Blues,"Documentary, Short","Prasanna, Prasenjit, Ayushi and Vivek are indi...",Vinayakram C,NaN,NaN
20831,tt30182426,Mottled Wood Owl,"Drama, Mystery","Arun, a young man, is on his way to railway tr...",Jinesh P K,"Roopesh T, Madhusoodan V","malayalamshortfilm, kuttichoolan"
20832,tt33513689,Skyward,"Short, Fantasy","Set against a surreal landscape, this is a tal...",Suruchi Sharma,"Jeevan Deni, Subrata Parashar, Gavari Devi Rao",NaN


In [215]:
new_dataframe['keywords'] = new_dataframe[['keywords']].apply(lambda x: ', '.join([str(i) for i in x if pd.notna(i)]), axis=1)

In [216]:
new_dataframe = new_dataframe.dropna(subset=['movie_id', 'movie_name','genre','overview','director','cast'])

In [217]:
new_dataframe.isnull().sum()

movie_id      0
movie_name    0
genre         0
overview      0
director      0
cast          0
keywords      0
dtype: int64

In [218]:
new_dataframe.shape

(19534, 7)

In [219]:
new_dataframe = new_dataframe.copy()
new_dataframe['cast'] = new_dataframe['cast'].str.replace(' ', '')
new_dataframe['genre'] = new_dataframe['genre'].str.replace(' ', '')
new_dataframe['director'] = new_dataframe['director'].str.replace(' ', '')
new_dataframe['keywords'] = new_dataframe['keywords'].str.replace(' ', '')

In [220]:
new_dataframe.loc[:, 'overview'] = new_dataframe['overview'].apply(lambda x: x.split())
new_dataframe['overview']

0        [A, high-octane, action, thriller, which, outl...
1        [A, single, mother, and, her, daughter, who, c...
2        [A, retired, jailer, goes, on, a, manhunt, to,...
3        [Flamboyant, Punjabi, Rocky, and, intellectual...
4        [An, unhappy, civilian, asks, the, court, to, ...
                               ...                        
20827    [A, filmic, letter, directed, by, Leo, Paapam,...
20828    [film, about, the, situation, of, that, Girls,...
20829    [Film, director, Tara, discovers, her, sister,...
20831    [Arun,, a, young, man,, is, on, his, way, to, ...
20832    [Set, against, a, surreal, landscape,, this, i...
Name: overview, Length: 19534, dtype: object

In [221]:
new_dataframe['genre'] = new_dataframe['genre'].apply(lambda x:x.split())
new_dataframe['director'] = new_dataframe['director'].apply(lambda x:x.split())
new_dataframe['cast'] = new_dataframe['cast'].apply(lambda x:x.split())
new_dataframe['keywords'] = new_dataframe['keywords'].apply(lambda x:x.split())

In [222]:
new_dataframe

,movie_id,movie_name,genre,overview,director,cast,keywords
0,tt15354916,Jawan,"[Action,Thriller]","[A, high-octane, action, thriller, which, outl...",[Atlee],"[ShahRukhKhan,Nayanthara,VijaySethupathi,Deepi...","[revenge,chase,suicide,farmer,train,injustice,..."
1,tt15748830,Jaane Jaan,"[Crime,Drama,Mystery]","[A, single, mother, and, her, daughter, who, c...",[SujoyGhosh],"[KareenaKapoor,JaideepAhlawat,VijayVarma,Saura...","[singlemother,police,investigation,criminal,ba..."
2,tt11663228,Jailer,"[Action,Comedy,Crime]","[A, retired, jailer, goes, on, a, manhunt, to,...",[NelsonDilipkumar],"[Rajinikanth,Mohanlal,Shivarajkumar,JackieShroff]","[prison,excop,intermission,actionhero,onewordt..."
3,tt14993250,Rocky Aur Rani Kii Prem Kahaani,"[Comedy,Drama,Family]","[Flamboyant, Punjabi, Rocky, and, intellectual...",[KaranJohar],"[RanveerSingh,AliaBhatt,Dharmendra,ShabanaAzmi]","[love,familyrelationships,romcom,newsreporter,..."
4,tt15732324,OMG 2,"[Comedy,Drama]","[An, unhappy, civilian, asks, the, court, to, ...",[AmitRai],"[PankajTripathi,AkshayKumar,YamiGautam,PawanMa...","[courtroom,sexeducation,courtroomdrama,referen..."
...,...,...,...,...,...,...,...
20827,tt32333612,Dear Sahithya,"[Short,Documentary]","[A, filmic, letter, directed, by, Leo, Paapam,...",[LeoPaapam],"[Sahithya,AbarnaSethuraman,ChinmayBane]",[]
20828,tt33067714,GHOOR,[Drama],"[film, about, the, situation, of, that, Girls,...",[HarshGogi],"[VandanaSandhu,JasvirJassi,VivekSharma,NareshN...","[pairofevileyes,shortstory,younggirls,punjabim..."
20829,tt32623558,Black Coffee,[Drama],"[Film, director, Tara, discovers, her, sister,...",[KhanAamir],"[RichaSinghChauhan,MamtaJain,SachinVidrohi,San...",[]
20831,tt30182426,Mottled Wood Owl,"[Drama,Mystery]","[Arun,, a, young, man,, is, on, his, way, to, ...",[JineshPK],"[RoopeshT,MadhusoodanV]","[malayalamshortfilm,kuttichoolan]"


In [223]:
new_dataframe['tags'] = new_dataframe['overview'] + new_dataframe['genre'] + new_dataframe['director'] + new_dataframe['cast'] + new_dataframe["keywords"]

In [224]:
new_dataframe = new_dataframe.drop(columns=['overview','genre','cast','director', 'keywords'])

In [225]:
new_dataframe.head()

,movie_id,movie_name,tags
0,tt15354916,Jawan,"[A, high-octane, action, thriller, which, outl..."
1,tt15748830,Jaane Jaan,"[A, single, mother, and, her, daughter, who, c..."
2,tt11663228,Jailer,"[A, retired, jailer, goes, on, a, manhunt, to,..."
3,tt14993250,Rocky Aur Rani Kii Prem Kahaani,"[Flamboyant, Punjabi, Rocky, and, intellectual..."
4,tt15732324,OMG 2,"[An, unhappy, civilian, asks, the, court, to, ..."


In [226]:
new_dataframe['tags'] = new_dataframe['tags'].apply(lambda x: " ".join(x))
new_dataframe.head()

,movie_id,movie_name,tags
0,tt15354916,Jawan,A high-octane action thriller which outlines t...
1,tt15748830,Jaane Jaan,A single mother and her daughter who commit a ...
2,tt11663228,Jailer,A retired jailer goes on a manhunt to find his...
3,tt14993250,Rocky Aur Rani Kii Prem Kahaani,Flamboyant Punjabi Rocky and intellectual Beng...
4,tt15732324,OMG 2,An unhappy civilian asks the court to mandate ...


In [227]:
new_dataframe['tags'] = new_dataframe['tags'].str.replace(',', ' ')

In [228]:
new_dataframe['tags'][5]

'Much to the dismay of her husband  a middle-class housewife  known for her rebellious attitude in high school  decides to attend her class reunion. Drama SonalJoshi ShilpaShettyKundra AmitSadh ChaitannyaChoudhry KushaKapila'

In [229]:
new_dataframe['tags'] = new_dataframe['tags'].str.lower()

In [230]:
new_dataframe['movie_name'] = new_dataframe['movie_name'].str.lower()

In [231]:
new_dataframe.sample(10)

,movie_id,movie_name,tags
16267,tt7838260,paddayi,based on macbeth paddayi is the story of madh...
15706,tt6080914,dora,a father and a daughter buy a used car which i...
11321,tt0471571,athadu,a professional killer is hired to fake the ass...
14748,tt4897908,calendar girls,the film focuses on five girls who hail from d...
10215,tt0272001,bindhaast,mayu and vaiju decide to play a prank on their...
1331,tt15831978,cash,a young lady leave her suitcase at the ernakul...
14968,tt5201164,care of footpath 2,an emotional suspense thriller inspired by tru...
1121,tt10305616,malaal,malaal is the story of shiva and astha two ve...
17882,tt10801196,pressure cooker,a young man faces immense pressure from his fa...
20109,tt26235062,rangabali,shaurya aka show a self-centered & attention ...


In [232]:
new_dataframe.reset_index(drop=True, inplace=True)


In [233]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [234]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=3000,stop_words='english')


In [235]:
vector = cv.fit_transform(new_dataframe['tags']).toarray()

In [236]:
vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [237]:
vector.shape

(19534, 3000)

In [238]:
similarity = cosine_similarity(vector)

In [239]:
similarity

array([[1.        , 0.05954913, 0.18659112, ..., 0.        , 0.08269768,
        0.029173  ],
       [0.05954913, 1.        , 0.08703883, ..., 0.03187884, 0.07715167,
        0.04082483],
       [0.18659112, 0.08703883, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.03187884, 0.        , ..., 1.        , 0.11805627,
        0.03123475],
       [0.08269768, 0.07715167, 0.        , ..., 0.11805627, 1.        ,
        0.07559289],
       [0.029173  , 0.04082483, 0.        , ..., 0.03123475, 0.07559289,
        1.        ]])

In [240]:
def recommend(movie):
    index = new_dataframe[new_dataframe['movie_name'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[0:20]:
        #if i[1] >= 0.3:
            print(new_dataframe.iloc[i[0]].movie_name)      

In [242]:
recommend('jailer')

jailer
jeevan ki shatranj
pakka commercial
ek se badhkar ek
nyayam meere cheppali
badass ravikumar
animal
rifle club
ek saath do do
haddhu ledhu raa
karmayogam (return gift)
batla house
ek aur ek gyarah: by hook or by crook
achcham yenbadhu madamaiyada
parvarish
veer
ventilator
thekku vadakku
mufti
om


In [243]:
pickle.dump(new_dataframe,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))

In [106]:
new_dataframe.sample(5)

,movie_id,movie_name,tags
13203,tt3575136,nimirndhu nil,aravind a principled young man takes on the ...
720,tt0116950,maachis,a realistic hard-hitting portrayal of terrori...
3104,tt29137349,pithala maathi,unknown action drama romance manikavidya devad...
17405,tt11772644,backpackers,based on a true story backpackers follows the...
15598,tt7699198,oru kuprasidha payyan,it is a murder mystery which draws its inspira...


In [177]:
# Step 1: Tokenize the text for Doc2Vec
new_dataframe['tokens'] = new_dataframe['tags'].apply(lambda x: x.split())

In [178]:
# Step 2: Create tagged documents for each movie (Doc2Vec requires tags for documents)
tagged_data = [TaggedDocument(words=row['tokens'], tags=[str(i)]) for i, row in new_dataframe.iterrows()]

In [179]:
# Step 3: Train a Doc2Vec model
model_d2v = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4, epochs=20)
model_d2v.build_vocab(tagged_data)
model_d2v.train(tagged_data, total_examples=model_d2v.corpus_count, epochs=model_d2v.epochs)


In [180]:
# Step 3: Train a Doc2Vec model
model_d2v = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4, epochs=20)
model_d2v.build_vocab(tagged_data)
model_d2v.train(tagged_data, total_examples=model_d2v.corpus_count, epochs=model_d2v.epochs)

In [181]:
# Step 4: Generate movie embeddings using the trained Doc2Vec model
movie_embeddings = np.array([
    model_d2v.dv[str(i)] if str(i) in model_d2v.dv else np.zeros(model_d2v.vector_size)
    for i in range(len(new_dataframe))
])

In [182]:
similarity2 = cosine_similarity(movie_embeddings)

In [183]:
def recommend2(movie):
    index = new_dataframe[new_dataframe['movie_name'] == movie].index[0]
    distances = sorted(list(enumerate(similarity2[index])),reverse=True,key = lambda x: x[1])
    for i in distances[0:20]:
        if i[1] >= 0.3:
            print(new_dataframe.iloc[i[0]].movie_name)

In [ ]:
recommend2('jawan')